GITHUB_Link https://github.com/Alaa-Muhsin/Python_Projects

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import xlsxwriter

Task 1
---------
Scrap the USD To EGP Exchange rate from this website
https://www.xe.com/currencyconverter/convert/?Amount=1&From=USD&To=EGP
and then use it to make a software that takes amount of USD Dollars from the user and calculate how much will it cost in EGP.

In [2]:

res = requests.get("https://www.xe.com/currencyconverter/convert/?Amount=1&From=USD&To=EGP")
soup  = BeautifulSoup(res.text,"html.parser")
egy_pound_tag = soup.find("p",attrs={"class":"sc-b5d1d4ba-1 bPeOTN"})
egy_pound_tag.text
egy_pound = egy_pound_tag.text[:5]
egy_pound_float = float(egy_pound)
egy_pound_float

50.83

In [3]:
while True:
        user_Choice = input("Hello \n Please Choose : 1- Converting Currency From Dollar To EGP Pound . \n2- Converting Currency From EGP Pound To Dollar. \n3- Exit")
        #Converting From Dollar To EGP Pound
        if user_Choice == "1" :
            try:
                dollar = float(input("Please Enter Dollar amounts"))
                egy_pound_result = egy_pound_float * dollar
                print(f"{dollar} $ = {egy_pound_result} EGP Pound")
                
            except ValueError:
                print("Invalid input")
                
        #Converting From EGP Pound To Dollar
        elif user_Choice == "2" :
            try:
                egy_pound = float(input("Please Enter EGP Pounds amounts"))
                dollar_result = egy_pound / egy_pound_float
                print(f"{egy_pound} EGP Pound = {dollar_result} $")
                
            except ValueError:
                print(f"{user_Choice} is Invalid input")
                
        elif user_Choice == "3":
             break
        
        else:
             print(f"{user_Choice} is Invalid input")




100.0 $ = 5083.0 EGP Pound
B is Invalid input
10166.0 EGP Pound = 200.0 $
Invalid input


Task 2
--------- 
Scraping from forecast weather
Imagine you work for forecast weather Now imagine your boss wants you to find the temperture for each day in celsius degree. How could you do this with Beautiful Soup?
https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168

In [4]:
res = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup  = BeautifulSoup(res.text,"html.parser")
with open("forecasting_weather.csv", mode='w', encoding='utf-8') as fd:
    #Header
    fd.write("Period,Temperature_C,Temperature_F,Description\n")

    #Current Weather
    current_temperature_C = soup.find("p",attrs={"class":"myforecast-current-sm"}).text[:2]
    current_temperature_F = soup.find("p",attrs={"class":"myforecast-current-lrg"}).text[:2]
    forecast_tags_current_weather = soup.find("div",attrs={"class":"tombstone-container"}).find_all("p")
    current_period = forecast_tags_current_weather[0].text.strip("\n ")
    current_description = forecast_tags_current_weather[2].text

    #First_Row = Current Weather 
    fd.write(f"\"{current_period}\",\"{current_temperature_C}\"°C,\"{current_temperature_F}\"°F,\"{current_description}\"\n")

    forecast_tags = soup.find_all("div",attrs={"class":"tombstone-container"})
    for i in range(1,len(forecast_tags)):
        forecast_imformation = forecast_tags[i].find_all("p")
        period_name = forecast_imformation[0].text
        temperature = forecast_imformation[2].text.split(" ")
        temperature_in_F = float(temperature[1])
        temperature_in_C = round((temperature_in_F - 32) * 5/9 ,2)

        Description = forecast_imformation[3].text
        fd.write(f"\"{period_name}\",\"{temperature_in_C}\"°C,\"{temperature_in_F}\"°F,\"{Description}\"\n")





In [5]:
df = pd.read_csv("forecasting_weather.csv")
df


,Period,Temperature_C,Temperature_F,Description
0,New Year's Day,12°C,53°F,High: 58 °F
1,Tonight,8.89°C,48.0°F,Partly Cloudy
2,Thursday,16.67°C,62.0°F,Mostly Sunny
3,Thursday Night,10.0°C,50.0°F,Partly Cloudythen ChanceRain
4,Friday,15.56°C,60.0°F,Rain Likely
5,Friday Night,8.89°C,48.0°F,Chance Rainthen SlightChance Rain
6,Saturday,14.44°C,58.0°F,Mostly Sunny
7,Saturday Night,8.33°C,47.0°F,Mostly Cloudy
8,Sunday,15.56°C,60.0°F,Mostly Sunny


Task 3
---------
Scrap the books (name, price, rate) for each category and put them into a CSV & Excel file
https://books.toscrape.com/ 



(Without Category - Without Requesting each book details link)

In [6]:

page_url = "https://books.toscrape.com/catalogue/page-1.html"
res_books = requests.get(page_url)
soup_book  = BeautifulSoup(res_books.text,"html.parser")

#number of pages 
page_number = int(soup_book.find("li",attrs={"class":"current"}).text.strip("\n ").split("of ")[1])



with open("Books.csv", mode='w', encoding='utf-8') as fd:
  fd.write("Name_in_books_page,Price,Rating,Availability,Link,Image\n") # header

  for p in range(page_number):
    page_url = "https://books.toscrape.com/catalogue/page-"+ str(p) + ".html"

    res_books = requests.get(page_url)
    soup_book  = BeautifulSoup(res_books.text,"html.parser")

    book_tag = soup_book.find_all("li",attrs={"class":"col-xs-6 col-sm-4 col-md-3 col-lg-3"})
    length_tag =  len(book_tag)

    for i in range(len(book_tag)):

      #Book Link
      book_details_link = "https://books.toscrape.com/catalogue/" + book_tag[i].find("div").find("a").attrs['href']

      #book image 
      book_image = book_tag[i].find("img").attrs["src"]
      book_image = "https://books.toscrape.com/" + book_image[2:]

      #book name
      book_name = book_tag[i].find("h3").find("a").text

      #book Price
      book_price = book_tag[i].find("p",attrs={"class":"price_color"}).text[2:]

      #book Availability 
      book_availability = book_tag[i].find("p",attrs={"class":"instock availability"}).text.strip("\n ")

      book_rating = book_tag[i].find("p").attrs["class"][1]


      fd.write(f"\"{book_name}\",\"{book_price}\",\"{book_rating}\",\"{book_availability}\",\"{book_details_link}\",\"{book_image}\"\n")



In [7]:
df = pd.read_csv("Books.csv")
 
# saving xlsx file
df_xlsx = pd.ExcelWriter('Books.xlsx')
df.to_excel(df_xlsx, index=False)
df_xlsx.close()



In [10]:
df = pd.read_csv("Books.csv")
df

,Name_in_books_page,Price,Rating,Availability,Link,Image
0,A Light in the ...,51.77,Three,In stock,https://books.toscrape.com/catalogue/a-light-i...,https://books.toscrape.com//media/cache/2c/da/...
1,Tipping the Velvet,53.74,One,In stock,https://books.toscrape.com/catalogue/tipping-t...,https://books.toscrape.com//media/cache/26/0c/...
2,Soumission,50.10,One,In stock,https://books.toscrape.com/catalogue/soumissio...,https://books.toscrape.com//media/cache/3e/ef/...
3,Sharp Objects,47.82,Four,In stock,https://books.toscrape.com/catalogue/sharp-obj...,https://books.toscrape.com//media/cache/32/51/...
4,Sapiens: A Brief History ...,54.23,Five,In stock,https://books.toscrape.com/catalogue/sapiens-a...,https://books.toscrape.com//media/cache/be/a5/...
...,...,...,...,...,...,...
975,Icing (Aces Hockey #2),40.44,Four,In stock,https://books.toscrape.com/catalogue/icing-ace...,https://books.toscrape.com//media/cache/8d/1e/...
976,"Hawkeye, Vol. 1: My ...",45.24,Three,In stock,https://books.toscrape.com/catalogue/hawkeye-v...,https://books.toscrape.com//media/cache/16/46/...
977,Having the Barbarian's Baby ...,34.96,Four,In stock,https://books.toscrape.com/catalogue/having-th...,https://books.toscrape.com//media/cache/f4/83/...
978,"Giant Days, Vol. 1 ...",56.76,Four,In stock,https://books.toscrape.com/catalogue/giant-day...,https://books.toscrape.com//media/cache/35/0b/...


With Category recommend NOT to run as it takes 10 min estimated time

In [8]:

page_url = "https://books.toscrape.com/catalogue/page-1.html"
res_books = requests.get(page_url)
soup_book  = BeautifulSoup(res_books.text,"html.parser")

#number of pages 
page_number = int(soup_book.find("li",attrs={"class":"current"}).text.strip("\n ").split("of ")[1])



with open("Books_details.csv", mode='w', encoding='utf-8') as fd:
  fd.write("Full_Name,Name_in_books_page,Categoty,Price,Rating,Availability,Books_in_Stock,Link,Image\n") # header

  for p in range(page_number):
    page_url = "https://books.toscrape.com/catalogue/page-"+ str(p) + ".html"

    res_books = requests.get(page_url)
    soup_book  = BeautifulSoup(res_books.text,"html.parser")

    book_tag = soup_book.find_all("li",attrs={"class":"col-xs-6 col-sm-4 col-md-3 col-lg-3"})
    length_tag =  len(book_tag)

    for i in range(len(book_tag)):

      #Book Link
      book_details_link = "https://books.toscrape.com/catalogue/" + book_tag[i].find("div").find("a").attrs['href']

      #book image 
      book_image = book_tag[i].find("img").attrs["src"]
      book_image = "https://books.toscrape.com/" + book_image[2:]

      #book name
      book_name = book_tag[i].find("h3").find("a").text

      #book Price
      book_price = book_tag[i].find("p",attrs={"class":"price_color"}).text[2:]

      #book Availability 
      book_availability = book_tag[i].find("p",attrs={"class":"instock availability"}).text.strip("\n ")

      #book rating
      book_rating = book_tag[i].find("p").attrs["class"][1]

      #open book page 
      res_book_detail = requests.get(book_details_link)
      soup_book_detail  = BeautifulSoup(res_book_detail.text,"html.parser")

      #book full name from book page
      book_full_name = soup_book_detail.find("h1").text

      #book Category
      category_name=soup_book_detail.find("ul",attrs={"class":"breadcrumb"}).find_all("li")[2].text.strip("\n")

      #number of bokks available in the stock
      books_in_stock = soup_book_detail.find("p",attrs={"class":"instock availability"}).text.strip("\n ").split("(")[1].split(" ")[0]

      fd.write(f"\"{book_full_name}\",\"{book_name}\",\"{category_name}\",\"{book_price}\",\"{book_rating}\",\"{book_availability}\",\"{books_in_stock}\",\"{book_details_link}\",\"{book_image}\"\n")



Task 4
---------
once you decided to do some exploring an want to know each country's (name, capital, population, Area(km^2) 
scrape the 250 countries (name, capital, population, area) and save them into a csv file
use this site to scrape this data : https://www.scrapethissite.com/pages/simple/

In [11]:
res_countries = requests.get("https://www.scrapethissite.com/pages/simple/")
soup_countries  = BeautifulSoup(res_countries.text,"html.parser")
countries_tag = soup_countries.find_all("div",attrs={"class":"col-md-4 country"})
num_countries = len(countries_tag)

workbook = xlsxwriter.Workbook('Countries_write.xlsx')
worksheet = workbook.add_worksheet()
col = 0
#Excel Headder 
worksheet.write(0,col,"Country")
worksheet.write(0,col+1,"Capital")
worksheet.write(0,col+2,"Population")
worksheet.write(0,col+3,"Area")

with open("Countries.csv",mode='w',encoding='utf-8') as fd:
  fd.write("Country,Capital,Population,Area\n") # header
  for i in range(num_countries):

    col = 0
    country_name = countries_tag[i].find("h3").text.strip("\n ")
    country_capital = countries_tag[i].find("span",attrs={"class":"country-capital"}).text
    country_population = countries_tag[i].find("span",attrs={"class":"country-population"}).text
    country_area = countries_tag[i].find("span",attrs={"class":"country-area"}).text
    fd.write(f"\"{country_name}\",\"{country_capital}\",\"{country_population}\",\"{country_area}\"\n")
    worksheet.write(i+1,col,country_name)
    worksheet.write(i+1,col+1,country_capital)
    worksheet.write(i+1,col+2,country_population)
    worksheet.write(i+1,col+3,country_area)
    
 
workbook.close()



In [12]:
df = pd.read_csv("Countries.csv")
 
# saving xlsx file
df_xlsx = pd.ExcelWriter('Countries.xlsx')
df.to_excel(df_xlsx, index=False)
df_xlsx.close()


 

Task 5
---------
scraping data about hockey teams:
you have to scrap the first page table's data (Team name, year, wins, losses, win%, goals for(gf), golaagainst(ga)
from this website: https://www.scrapethissite.com/pages/forms/?page_num=1

In [13]:
res_hockey = requests.get("https://www.scrapethissite.com/pages/forms/?page_num=1")
soup_hockey  = BeautifulSoup(res_hockey.text,"html.parser")

hockey_table = soup_hockey.find("table",attrs={"class":"table"})
hockey_data = {}
#header ( dynamic header )
table_header = []
hockey_data[1]=["M"]
hockey_header = hockey_table.find("tr").find_all("th")
with open("Hockey.csv",mode='w',encoding='utf-8') as fd:
   for i in range (len(hockey_header)):
      table_header.append(hockey_header[i].text.strip("\n "))
      fd.write(f"\"{table_header[i]}\",")

   fd.write("\n")

   hockey_num_pages = soup_hockey.find("ul",attrs={"class":"pagination"}).find_all("li")
   hockey_num_pages = len(hockey_num_pages)
   
   for p in range (hockey_num_pages):
      hockey_URL = "https://www.scrapethissite.com/pages/forms/?page_num=" + str(p)
      res_hockey = requests.get(hockey_URL)
      soup_hockey  = BeautifulSoup(res_hockey.text,"html.parser")
      hockey_table = soup_hockey.find("table",attrs={"class":"table"})

      hockey_row = hockey_table.find_all("tr")

      for r in range(1,len(hockey_row)):
         hockey_data[r] = [ hockey_row[r].find_all("td")[0].text.strip("\n ") ]
         for col in range (1,len(hockey_header)):
            hockey_data[r] = hockey_data[r] + [hockey_row[r].find_all("td")[col].text.strip("\n ")]
   
      for r in range (1,len(hockey_row)):
         for c in range(len(hockey_header)):
            fd.write(f"\"{hockey_data[r][c]}\",")
         
         fd.write("\n")
   




Task 6
---------
Use this web service 
https://jsonplaceholder.typicode.com/users
and make a program that get all users data including name, username, email, street, suite, city, zip code, geo_lat & geo_long and then save them in a CSV & Excel file.

In [14]:
users_data_link = "https://jsonplaceholder.typicode.com/users"
res_users = requests.get(users_data_link)
res_users = res_users.json()
with open("Users_Data.csv", mode='w', encoding='utf-8') as fd:
    #Header
    fd.write("ID,Name,User_Name,Email,Street,Suite,City,Zipcode,Geo_Lat,Geo_Lng,Phone,Website,Company_Name,Company_CatchPhrase,Company_Bs\n")
    for i in range(len(res_users)):
        user_id = res_users[i]["id"]
        user_name = res_users[i]["name"]
        user_username = res_users[i]["username"]
        user_email = res_users[i]["email"]
        user_street = res_users[i]["address"]["street"]
        user_suite = res_users[i]["address"]["suite"]
        user_city = res_users[i]["address"]["city"]
        user_zipcode = res_users[i]["address"]["zipcode"]
        user_geo_lat = res_users[i]["address"]["geo"]["lat"]
        user_geo_lng = res_users[i]["address"]["geo"]["lng"]
        user_phone = res_users[i]["phone"]
        user_website = res_users[i]["website"]
        user_company_name = res_users[i]["company"]["name"]
        user_company_catchPhrase = res_users[i]["company"]["catchPhrase"]
        user_company_bs  = res_users[i]["company"]["bs"]
      
        fd.write(f"\"{user_id}\",\"{user_name}\",\"{user_username}\",\"{user_email}\",\"{user_street}\",\"{user_suite}\",\"{user_city}\",\"{user_zipcode}\",\"{user_geo_lat}\",\"{user_geo_lng}\",\"{user_phone}\",\"{user_website}\",\"{user_company_name}\",\"{user_company_catchPhrase}\",\"{user_company_bs}\"\n")


In [15]:
df = pd.read_csv("Users_Data.csv")
df


,ID,Name,User_Name,Email,Street,Suite,City,Zipcode,Geo_Lat,Geo_Lng,Phone,Website,Company_Name,Company_CatchPhrase,Company_Bs
0,1,Leanne Graham,Bret,Sincere@april.biz,Kulas Light,Apt. 556,Gwenborough,92998-3874,-37.3159,81.1496,1-770-736-8031 x56442,hildegard.org,Romaguera-Crona,Multi-layered client-server neural-net,harness real-time e-markets
1,2,Ervin Howell,Antonette,Shanna@melissa.tv,Victor Plains,Suite 879,Wisokyburgh,90566-7771,-43.9509,-34.4618,010-692-6593 x09125,anastasia.net,Deckow-Crist,Proactive didactic contingency,synergize scalable supply-chains
2,3,Clementine Bauch,Samantha,Nathan@yesenia.net,Douglas Extension,Suite 847,McKenziehaven,59590-4157,-68.6102,-47.0653,1-463-123-4447,ramiro.info,Romaguera-Jacobson,Face to face bifurcated interface,e-enable strategic applications
3,4,Patricia Lebsack,Karianne,Julianne.OConner@kory.org,Hoeger Mall,Apt. 692,South Elvis,53919-4257,29.4572,-164.2990,493-170-9623 x156,kale.biz,Robel-Corkery,Multi-tiered zero tolerance productivity,transition cutting-edge web services
4,5,Chelsey Dietrich,Kamren,Lucio_Hettinger@annie.ca,Skiles Walks,Suite 351,Roscoeview,33263,-31.8129,62.5342,(254)954-1289,demarco.info,Keebler LLC,User-centric fault-tolerant solution,revolutionize end-to-end systems
5,6,Mrs. Dennis Schulist,Leopoldo_Corkery,Karley_Dach@jasper.info,Norberto Crossing,Apt. 950,South Christy,23505-1337,-71.4197,71.7478,1-477-935-8478 x6430,ola.org,Considine-Lockman,Synchronised bottom-line interface,e-enable innovative applications
6,7,Kurtis Weissnat,Elwyn.Skiles,Telly.Hoeger@billy.biz,Rex Trail,Suite 280,Howemouth,58804-1099,24.8918,21.8984,210.067.6132,elvis.io,Johns Group,Configurable multimedia task-force,generate enterprise e-tailers
7,8,Nicholas Runolfsdottir V,Maxime_Nienow,Sherwood@rosamond.me,Ellsworth Summit,Suite 729,Aliyaview,45169,-14.3990,-120.7677,586.493.6943 x140,jacynthe.com,Abernathy Group,Implemented secondary concept,e-enable extensible e-tailers
8,9,Glenna Reichert,Delphine,Chaim_McDermott@dana.io,Dayna Park,Suite 449,Bartholomebury,76495-3109,24.6463,-168.8889,(775)976-6794 x41206,conrad.com,Yost and Sons,Switchable contextually-based project,aggregate real-time technologies
9,10,Clementina DuBuque,Moriah.Stanton,Rey.Padberg@karina.biz,Kattie Turnpike,Suite 198,Lebsackbury,31428-2261,-38.2386,57.2232,024-648-3804,ambrose.net,Hoeger LLC,Centralized empowering task-force,target end-to-end models
